In [ ]:
%pip install geopandas matplotlib plotly nbformat notebook ipywidgets ipyleaflet

In [1]:
import geopandas as gpd
import pandas as pd
from ipyleaflet import Map, GeoJSON, WidgetControl
import ipywidgets as widgets

# Load shapefile
world = gpd.read_file("./data/ne_110m_admin_0_countries/ne_110m_admin_0_countries.shp")
europe = world[world['CONTINENT'] == 'Europe']

# Load OWID CO₂ data
co2_df = pd.read_csv("https://github.com/owid/co2-data/raw/master/owid-co2-data.csv")

# Define a list of European countries
european_countries = [
    'Albania', 'Andorra', 'Armenia', 'Austria', 'Azerbaijan', 'Belarus', 'Belgium',
    'Bosnia and Herzegovina', 'Bulgaria', 'Croatia', 'Cyprus', 'Czechia', 'Denmark',
    'Estonia', 'Finland', 'France', 'Georgia', 'Germany', 'Greece', 'Hungary',
    'Iceland', 'Ireland', 'Italy', 'Kazakhstan', 'Kosovo', 'Latvia', 'Liechtenstein',
    'Lithuania', 'Luxembourg', 'Malta', 'Moldova', 'Monaco', 'Montenegro',
    'Netherlands', 'North Macedonia', 'Norway', 'Poland', 'Portugal', 'Romania',
    'Russia', 'San Marino', 'Serbia', 'Slovakia', 'Slovenia', 'Spain', 'Sweden',
    'Switzerland', 'Turkey', 'Ukraine', 'United Kingdom', 'Vatican'
]

# Filter for European countries and years of interest
europe_emissions_raw = co2_df[
    (co2_df['country'].isin(european_countries)) &
    (co2_df['year'].isin([2019, 2021]))
]

# Pivot to wide format: one row per country
emissions_df = europe_emissions_raw.pivot(index='country', columns='year', values='co2').reset_index()
emissions_df.columns = ['ADMIN', 'co2_2019', 'co2_2021']
emissions_df = emissions_df.fillna(0)

# Optional country name fixes (match shapefile naming)
country_name_map = {
    "Czechia": "Czech Republic",
    "Bosnia and Herzegovina": "Bosnia and Herz.",
    "North Macedonia": "Macedonia",
    "Slovakia": "Slovak Republic",
    "Moldova": "Moldova",
    "Russia": "Russian Federation"
}
emissions_df['ADMIN'] = emissions_df['ADMIN'].replace(country_name_map)

# Merge with shapefile data
europe_emissions = europe.merge(emissions_df, how='left', on='ADMIN').fillna(0)

# Define GeoJSON styling
def get_geojson(year):
    def style_callback(feature):
        admin = feature['properties']['ADMIN']
        value = emissions_df.loc[emissions_df['ADMIN'] == admin, f'co2_{year}'].values
        emission = value[0] if len(value) else 0
        intensity = min(emission / emissions_df[f'co2_{year}'].max(), 1.0)
        color = f"rgba(255, 0, 0, {intensity})"
        return {
            'fillColor': color,
            'color': 'black',
            'weight': 1,
            'fillOpacity': 0.6,
        }

    return GeoJSON(
        data=europe_emissions.__geo_interface__,
        style_callback=style_callback,
        name=f"CO₂ {year}"
    )

# Create map and interactive controls
center = [54.0, 15.0]
m = Map(center=center, zoom=4)

layer_ref = [get_geojson("2019")]
m.add_layer(layer_ref[0])

dropdown = widgets.Dropdown(
    options=[('2019 (Pre-Pandemic)', '2019'), ('2021 (Post-Pandemic)', '2021')],
    value='2019',
    description='Year:',
)

def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        m.remove_layer(layer_ref[0])
        new_layer = get_geojson(change['new'])
        m.add_layer(new_layer)
        layer_ref[0] = new_layer

dropdown.observe(on_change)

control = WidgetControl(widget=dropdown, position='topright')
m.add_control(control)

m


/var/folders/7g/n_c_1w8j6nq8cmbcbnl7mpcc0000gn/T/ipykernel_5752/459343731.py:48: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  europe_emissions = europe.merge(emissions_df, how='left', on='ADMIN').fillna(0)


Map(center=[54.0, 15.0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out…

In [4]:
import pandas as pd

# Einwohnerzahlen (Beispielwerte, bitte ggf. anpassen!)
population = {
    "Austria": 8917205,
    "Belgium": 11589623,
    "Bulgaria": 6951482,
    "Croatia": 4047200,
    "Cyprus": 888005,
    "Czechia": 10708981,
    "Denmark": 5831404,
    "Estonia": 1326535,
    "Finland": 5540720,
    "France": 67391582,
    "Germany": 83166711,
    "Greece": 10374841,
    "Hungary": 9749763,
    "Iceland": 372520,
    "Ireland": 5020197,
    "Italy": 59554023,
    "Latvia": 1886198,
    "Lithuania": 2794700,
    "Luxembourg": 654768,
    "Malta": 519562,
    "Netherlands": 17441139,
    "Norway": 5421241,
    "Poland": 37751234,
    "Portugal": 10305564,
    "Romania": 19128707,
    "Slovakia": 5456362,
    "Slovenia": 2119675,
    "Spain": 47450795,
    "Sweden": 10515107,
    "Switzerland": 8740000
}

# CSV einlesen
df = pd.read_csv(
    "./data/assets/Co2_Stats.csv",
    sep='\t',
    encoding='utf-16'
)

# Emissionsspalte bereinigen und in float umwandeln
df['t CO2 equivalent'] = df['t CO2 equivalent'].str.replace('.', '', regex=False).astype(float)

# Pro-Kopf-Emission berechnen
df['co2_per_capita'] = df.apply(
    lambda row: row['t CO2 equivalent'] / population.get(row['Country'], 1) if population.get(row['Country']) else None,
    axis=1
)

# Neue CSV speichern
df.to_csv("./data/assets/Co2_Stats_per_capita.csv", sep='\t', index=False)

In [12]:
import geopandas as gpd
import pandas as pd
from ipyleaflet import Map, GeoJSON, WidgetControl
import ipywidgets as widgets

# Shapefile laden
world = gpd.read_file("./data/ne_110m_admin_0_countries/ne_110m_admin_0_countries.shp")
europe = world[world['CONTINENT'] == 'Europe']

# CO2 pro Kopf laden
co2_df = pd.read_csv("./data/assets/Co2_Stats_per_capita.csv", sep='\t', encoding='utf-8')
co2_df = co2_df.rename(columns={
    'Country': 'ADMIN',
    'Jahr von Date': 'year',
    'co2_per_capita': 'co2'
})

# Länder-Namen anpassen VOR dem Pivot!
country_name_map = {
    "Czechia": "Czech Republic",
    "Bosnia and Herzegovina": "Bosnia and Herz.",
    "North Macedonia": "Macedonia",
    "Slovakia": "Slovak Republic",
    "Russia": "Russian Federation",
    "United Kingdom": "United Kingdom",
    "Moldova": "Moldova",
    "Republic of Serbia": "Serbia",
    "Montenegro": "Montenegro",
    "Kosovo": "Kosovo",
    "Albania": "Albania",
    "Belarus": "Belarus",
    "Ukraine": "Ukraine"
}
co2_df['ADMIN'] = co2_df['ADMIN'].replace(country_name_map)

co2_df['year'] = co2_df['year'].astype(int)
co2_df['co2'] = co2_df['co2'].astype(float)

# Pivot: eine Zeile pro Land, Spalten für Jahre
emissions_df = co2_df.pivot(index='ADMIN', columns='year', values='co2').reset_index()

# Merge mit Shapefile
europe_emissions = europe.merge(emissions_df, how='left', on='ADMIN').fillna(0)

# GeoJSON-Style-Funktion
def get_geojson(year):
    def style_callback(feature):
        admin = feature['properties']['ADMIN']
        value = emissions_df.loc[emissions_df['ADMIN'] == admin, year].values
        emission = value[0] if len(value) else 0
        max_emission = emissions_df[year].max() if emissions_df[year].max() > 0 else 1
        intensity = min(emission / max_emission, 1.0)
        color = f"rgba(255, 0, 0, {intensity})"
        return {
            'fillColor': color,
            'color': 'black',
            'weight': 1,
            'fillOpacity': 0.6,
        }
    return GeoJSON(
        data=europe_emissions.__geo_interface__,
        style_callback=style_callback,
        name=f"CO₂ pro Kopf {year}"
    )

# Map und Dropdown
center = [54.0, 15.0]
m = Map(center=center, zoom=4)

# Jahre aus den Daten dynamisch holen
years = sorted([col for col in emissions_df.columns if isinstance(col, int)])

layer_ref = [get_geojson(years[0])]
m.add_layer(layer_ref[0])

dropdown = widgets.Dropdown(
    options=[(str(y), y) for y in years],
    value=years[0],
    description='Jahr:',
)

def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        m.remove_layer(layer_ref[0])
        new_layer = get_geojson(change['new'])
        m.add_layer(new_layer)
        layer_ref[0] = new_layer

dropdown.observe(on_change)

control = WidgetControl(widget=dropdown, position='topright')
m.add_control(control)

m

/var/folders/7g/n_c_1w8j6nq8cmbcbnl7mpcc0000gn/T/ipykernel_5752/2147606603.py:43: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  europe_emissions = europe.merge(emissions_df, how='left', on='ADMIN').fillna(0)


Map(center=[54.0, 15.0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out…

In [15]:
import geopandas as gpd
from ipyleaflet import Map, GeoJSON

# Shapefile laden
world = gpd.read_file("./data/ne_110m_admin_0_countries/ne_110m_admin_0_countries.shp")
europe = world[world['CONTINENT'] == 'Europe']

# Nur die Geodaten von Europa als GeoJSON-Layer anzeigen
center = [54.0, 15.0]
m = Map(center=center, zoom=4)

geojson = GeoJSON(
    data=europe.__geo_interface__,
    style={
        'color': 'blue',
        'weight': 1,
        'fillOpacity': 0.1
    },
    name="Europa"
)
m.add_layer(geojson)
m

Map(center=[54.0, 15.0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out…